<a href="https://colab.research.google.com/github/beifa/M5_Forecasting/blob/master/v203_fork_of_wrmsse_metric_predict_8b1065.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc
import pickle
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
import psutil, warnings
from multiprocessing import Pool
PATH = '/kaggle/input/wmt-inc/'
PATH_MODEL = '/kaggle/input/model-wmt-v3/'
features_saved = ''
SEED = 13
random.seed(SEED)
np.random.seed(SEED)
warnings.filterwarnings('ignore')
N_CORES = psutil.cpu_count()     # Available CPU cores
VER = 203
lgb_params['seed'] = SEED
stores = [
          'CA_1',
          'CA_2',
          'CA_3',
          'CA_4',
          'TX_1',
          'TX_2',
          'TX_3',
          'WI_1',
          'WI_2',
          'WI_3'
          ]

In [ ]:
# SPLIT = []
# roll = [7, 14, 30, 60]
# for i in roll:
#     for j in roll:
#         SPLIT.append([i, j])

#SPLITS for lags creation

#default
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        SPLIT.append([i,j])
        
        
#v1 ewm
# SHIFT_DAY  = 28
# N_LAGS     = 15
# LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
# SPLIT = []
# for i in [1, 5, 7, 10, 14, 19]:
#     for j in [7, 14, 21, 30, 37, 44, 51, 60]:
#         SPLIT.append([i,j])
    
        

## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d','sales']]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    #v1
#     lag_df[col_name] = lag_df.groupby(['id'])['sales'].transform(lambda x: x.shift(shift_day).ewm(span = roll_wind, adjust=False)\
#                                     .mean()).astype(np.float16)
    #v2
    lag_df[col_name] = lag_df.groupby(['id'])['sales'].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

def get_base_test():
    base_test = pd.DataFrame()
    for store_id in STORES_IDS:
        temp_df = pd.read_pickle(PATH_MODEL + 'test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    return base_test

base_test = get_base_test()

In [ ]:
all_preds = pd.DataFrame()

for d in range(1, 29):
    print(f'Predict day {d}')
    temp_df = base_test.copy()
    temp_df = pd.concat([temp_df, df_parallelize_run(make_lag_roll, SPLIT)], axis=1)
    #loop for stores for 1 day
    for store in stores:
        model_name = 'lgb_model_'+store+'_v'+str(VER)+'.bin' 
        estimator = pickle.load(open(PATH_MODEL + model_name, 'rb')) 
        #make mask
        day_mask = base_test['d'] == (1941  + d)
        store_mask = base_test['store_id'] == store
        mask = (day_mask) & (store_mask)    
        base_test['sales'][mask] = estimator.predict(temp_df[mask][features_saved])
    pred_df = base_test[day_mask][['id', 'sales']]
   
    pred_df.columns = ['id','F'+str(d)]
    if 'id' in list(all_preds): 
        all_preds = all_preds.merge(pred_df, on=['id'], how='left')
    else:
        all_preds = pred_df.copy()
    print(all_preds.shape)
    print(all_preds['F'+str(d)].sum())
    del pred_df
    del temp_df
    gc.collect()

In [ ]:
all_preds = all_preds.reset_index(drop=True)
all_preds

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.934082,0.778809,0.712402,0.837402,0.994629,1.156250,1.052734,1.028320,0.813965,...,1.001953,1.253906,1.168945,0.898926,0.770020,0.742676,0.812500,1.005859,1.212891,1.038086
1,HOBBIES_1_002_CA_1_evaluation,0.210205,0.187500,0.182739,0.187012,0.223511,0.289795,0.318359,0.221680,0.199707,...,0.260986,0.329834,0.343262,0.207764,0.208252,0.210693,0.235352,0.282959,0.347656,0.380371
2,HOBBIES_1_003_CA_1_evaluation,0.568359,0.472168,0.508301,0.544434,0.674805,0.836914,0.799316,0.548340,0.511230,...,0.596191,0.756348,0.687988,0.488770,0.449219,0.435791,0.506348,0.601562,0.702637,0.628906
3,HOBBIES_1_004_CA_1_evaluation,1.516602,1.353516,1.309570,1.376953,1.732422,2.365234,2.750000,1.931641,1.366211,...,1.827148,2.421875,2.910156,1.815430,1.403320,1.416992,1.471680,1.812500,2.517578,2.478516
4,HOBBIES_1_005_CA_1_evaluation,1.088867,0.946289,0.950195,1.047852,1.137695,1.386719,1.534180,1.258789,1.037109,...,1.258789,1.512695,1.601562,1.151367,1.060547,1.027344,1.062500,1.260742,1.539062,1.385742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.490234,0.452393,0.440430,0.490234,0.546387,0.557617,0.681641,0.519043,0.496338,...,0.580566,0.736816,0.873535,0.613770,0.585938,0.603516,0.493164,0.545898,0.551270,0.708496
30486,FOODS_3_824_WI_3_evaluation,0.258545,0.253174,0.229126,0.229370,0.238159,0.289795,0.316162,0.243774,0.240234,...,0.266113,0.375732,0.397949,0.274658,0.385742,0.351074,0.306885,0.260254,0.319580,0.335205
30487,FOODS_3_825_WI_3_evaluation,0.662109,0.563965,0.548828,0.508789,0.580566,0.728027,0.748047,0.711914,0.577148,...,0.767578,1.128906,1.291992,1.031250,1.048828,1.085938,0.739746,0.768555,0.860840,1.016602
30488,FOODS_3_826_WI_3_evaluation,1.060547,1.068359,1.080078,1.036133,1.101562,1.310547,1.286133,1.310547,1.101562,...,1.192383,1.617188,1.704102,1.251953,1.469727,1.385742,1.155273,1.192383,1.327148,1.366211


In [ ]:
# we need to do fillna() for "_evaluation" items

# submission = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')[['id']]
# submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
# sub = submission.iloc[:30490]
# pred_to_eval = sub.copy()
# pred_to_eval['id'] = pred_to_eval['id'].str.replace('_validation', '_evaluation')
# pred = pd.concat([sub, pred_to_eval], axis = 0)
# pred.to_csv('submission_v_eval'+str(VER)+'.csv', index=False)

In [ ]:
submission = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
sub = submission.iloc[30490:]
pred_to_val = sub.copy()
pred_to_val['id'] = pred_to_val['id'].str.replace('_evaluation', '_validation')
pred = pd.concat([pred_to_val, sub], axis = 0)
pred.to_csv('submission_v_eval'+str(VER)+'.csv', index=False)